### 主要对item_category_list, item_property_list, predict_category_property三个object属性进行处理

In [1]:
import time
import datetime
import gc
from collections import Counter

import pandas as pd
import numpy as np

In [ ]:
# 读取Step_4处理后的数据
data = pd.read_csv('../Temp/train_day_7_test_step_4.csv')

In [ ]:
# 从predicte_category_property中提取category和property

pred_cate_lists = []
pred_prop_lists = []

for i in range(len(data)):
    pred_cate_list = []
    pred_prop_list = []
    
    if data['predict_category_property'][i] == str(-1):
        pred_cate_lists.append('-1')
        pred_prop_lists.append('-1')
        continue
    else:
        pred_cate_props = data['predict_category_property'][i].split(';')# 分拆类别属性组
    
    for pred_cate_prop in pred_cate_props:# 考虑一个类别属性组
        pred_cate = pred_cate_prop.split(':')[0]# 提取类别
    
        if pred_cate not in pred_cate_list:# 只考虑不重复的类别，重复的不计入
            pred_cate_list.append(pred_cate)
        
        pred_props = pred_cate_prop.split(':')[1]
        # props外加中括号，否则会拆分一个字符串
        for pred_prop in [pred_props]: 
            if pred_prop not in pred_prop_list:# 只考虑不重复的属性，重复的不计入
                pred_prop_list.append(pred_prop)
    
    pred_cate_lists.append(pred_cate_list)
    pred_prop_lists.append(pred_prop_list)

In [ ]:
#整理样本的真实类别
cate_lists = []

for i in range(len(data)):
    cate = data['item_category_list'][i].split(';')
    cate_lists.append(cate)

In [ ]:
#计算预测类别的命中率

cate_proportion_list = []
for pred_cate, cate in zip(pred_cate_lists, cate_lists):
    count = 0
    for i in pred_cate:
        if i in cate:
            count += 1
    cate_proportion = count / len(pred_cate)
    cate_proportion_list.append(cate_proportion)

In [ ]:
#整理样本的真实属性

prop_lists = []

for i in range(len(data)):
    prop = data['item_property_list'][i].split(';')
    prop_lists.append(prop)

In [ ]:
#计算预测属性的命中率

prop_proportion_list = []
for pred_prop, prop in zip(pred_prop_lists, prop_lists):
    count = 0
    for i in pred_prop:
        if i in prop:
            count += 1
    prop_proportion = count / len(pred_prop)
    prop_proportion_list.append(prop_proportion)

In [ ]:
#把预测属性命中率、预测类别命中率二者同data融合

cate_proportion_list = pd.DataFrame(cate_proportion_list).rename(columns={0: 'cate_proportion_list'})
cate_proportion_list['instance_id'] = data['instance_id']
data = pd.merge(data, cate_proportion_list, how='left', on='instance_id')

prop_proportion_list = pd.DataFrame(prop_proportion_list).rename(columns={0: 'prop_proportion_list'})
prop_proportion_list['instance_id'] = data.instance_id
data = pd.merge(data, prop_proportion_list, how='left', on='instance_id')

In [ ]:
# 保存处理后的数据
data.to_csv('../Temp/train_day_7_test_step_5.csv', index=False)